In [2]:
!pip install -U kaggle-environments

Requirement already up-to-date: kaggle-environments in /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages (0.3.12)
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.6/bin/python3.6 -m pip install --upgrade pip' command.


In [ ]:
from kaggle_environments import make, evaluate
import numpy as np
import torch
import torch.nn.functional as F
from random import choice
print(np.__version__, torch.__version__)

In [ ]:
# Constants #
ROWS = 21
COlS = 21

# State #
init = False
size = 0
me = 0


# Map #
class Direction:
    NORTH = 1
    SOUTH = 2
    EAST = 3
    WEST = 4
    STATIONARY = 5
    
class MapLocation: 
    # Location
    x,y = 0,0 
    # Halite
    halite = 0
    # Unit
    unit = None
    
    def __init__(self,x,y,halite,unit):
        self.x,self.y,self.halite,self.unit = x,y,halite,unit
        
    def distanceTo(self,target):
        if type(target) is MapLocation:
            col = abs(self.x - target.x)
            col = min(col, COLS - col)
            row = abs(self.y - target.y)
            row = min(row_d, ROWS - row)
            return row + col
        
    def directionTo(self,target):
        res = Direction.STATIONARY
        if target.x != self.x:
            if target.x > self.x and target.x - self.x < COLS / 2:
                res = Direction.EAST
            elif self.x > target.x and self.x - target.x > COLS / 2:
                res = Direction.EAST
            else: 
                res = Direction.WEST
        elif target.y != self.y: 
            if target.y > self.y and target.y - self.y < ROWS / 2:
                res = Direction.NORTH
            elif self.y > target.y and self.y - target.y > ROWS / 2:
                res = Direction.NORTH
            else: 
                res = Direction.SOUTH
        return res

class Board:
    pass
            

# Units #

class Ship:
    action = {}
    x, y = 0, 0
    unitID = 0
    def __init__(self,action, unitID, x, y):
        self.action = action
        self.unitID = unitID
        self.x = x
        self.y = y

class Shipyard:
    action = {}
    x, y = 0, 0
    unitID = 0
    
    def __init__(self,action, unitID, x, y):
        self.action = action
        self.unitID = unitID
        self.x = x
        self.y = y
    

In [ ]:
def agent(obs, config):
    # parse obs
    global init, size, me
    if not init:
        me = obs.player
        size = config.size
        init = True
    halite = np.array(obs.halite).reshape(size, size)
    my_halite = obs.players[me][0]
    my_shipyards = obs.players[me][1]
    my_ships = obs.players[me][2]
    
    action = {}
    for shipyard, raw_pos in my_shipyards.items():
        x, y = raw_pos // size, raw_pos % size
        action[shipyard] = "SPAWN"
        
    for ship_id, (raw_pos, cargo) in my_ships.items():
        x, y = raw_pos // size, raw_pos % size
        if ship_id in ships:
            ship = ships[ship_id]
            # update observation
            ship.update(x, y, cargo, halite)
            action[ship_id].update(ship.decide())        

    return action

In [ ]:
env = make("halite", debug=True)
env.run([agent, "random"])
env.render(mode="ipython", width=800, height=600)

In [ ]:
import json
print("Configuration:", json.dumps(env.specification.configuration, indent=4, sort_keys=True))
print("Observation:", json.dumps(env.specification.observation, indent=4, sort_keys=True))
print("Action:", json.dumps(env.specification.action, indent=4, sort_keys=True))